### Query Decomposition:

Query decomposition is the process of takinga complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

It is useful in
* Complex queries often involve multiple concepts.
* LLMs or retrievers may miss parts of the original question.
* It enables multi-hop reasoning (answering in steps)
* Allows parallelism(especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

d:\RAGs In Depth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#Data Ingestion and Embedding Documents

documents = TextLoader('langchain_crewai.txt', encoding="utf-8").load()

chunks = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50).split_documents(documents)

embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

vectorstore= FAISS.from_documents(chunks, embeddings)

retriever= vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":4, "lambda_mult":0.7})

#The lambda_mult parameter controls this trade-off:

# lambda_mult = 1.0 → prioritize only relevance (like normal similarity search)
# lambda_mult = 0.0 → prioritize only diversity